This notebook contains a demonstration of the Fermi-GBM classification algorithm using triggered data. It uses the best-fit source position from the flight localization to determine the two detectors closest to the source.

**Disclaimer: the GDT implementation of the onboard classifier is a close, but not exact, approximation of the code running on the spacecraft. There may be small differences compared to triggers reported by the spacecraft.**

To begin, use the **TriggerFinder** class to download a trigdat file for a burst you'd like to analyze.

In [1]:
from gdt.missions.fermi.gbm.finders import TriggerFinder

# initialize finder with burst number 181010247 and download the trigdat file
finder = TriggerFinder("181010247")
trigdat_path = finder.get_trigdat(".")[0]

Output()

Next, open the **Trigdat** file

In [2]:
from gdt.missions.fermi.gbm.trigdat import Trigdat

trigdat = Trigdat.open(trigdat_path)

You can now pass the opened file to the Classification class, which performs an approximation of the source classification routine used on-board Fermi-GBM. During this step we apply the following settings:
* `use_loc=True` calculates the detectors are closest to the source location. When `False` the routine will instead sort the detectors by the largest count rates in the 50-300 keV energy band.
* `verbose=True` prints the trigdat information used as part of the classification algorithm. Setting this to `False` or ignoring this argument will silence this output.

In [3]:
from gdt.missions.fermi.gbm.classification import Classification

c = Classification()
det_nums = c.classify_trigdat(trigdat, use_loc=True, verbose=True)


Trigger Algorithm = 6
Location = (RA 54.867, Dec -22.333) deg
Location Err = 13.083 deg

Rates
 [[ 69. 303. 219. 171. 194.  58.  27.  79.]
 [ 50. 265. 232. 200. 195.  45.  40.  45.]
 [ 31. 149. 154. 171. 180.  39.  72.  45.]
 [141. 490. 275. 166. 130.  39.  24.  50.]
 [130. 527. 288. 170. 138.  35.  57.  39.]
 [ 85. 353. 185. 159. 161.  37.  63.  19.]
 [ 71. 340. 220. 217. 188.  28.  66.  21.]
 [104. 398. 246. 168. 156.  33.  36.  45.]
 [ 86. 416. 269. 193. 177.  24.  46.  30.]
 [ 76. 395. 292. 249. 250.  51.  97.  24.]
 [ 92. 291. 258. 235. 227.  44.  95.  23.]
 [ 66. 345. 229. 198. 191.  51.  39.  87.]
 [549. 386. 508. 210.  37.  11.  13.  77.]
 [484. 389. 410. 173.  36.  29.  10.  94.]]

Background
 [[ 66. 290. 208. 159. 166.  37.  34.  68.]
 [ 57. 271. 206. 165. 157.  37.  40.  59.]
 [ 33. 136. 160. 153. 163.  43.  66.  38.]
 [126. 496. 264. 178. 155.  34.  30.  57.]
 [130. 519. 266. 185. 152.  35.  54.  30.]
 [ 86. 322. 208. 157. 159.  40.  72.  23.]
 [ 69. 312. 204. 162. 143.  3

The `classify_trigdat()` method returns a list with the detector numbers for the top 2 detectors closest to the source location. Let's check which ones they are

In [4]:
from gdt.missions.fermi.gbm.detectors import GbmDetectors

print("\nClosest detectors:")
for num in det_nums:
    print("  {}".format(GbmDetectors.from_num(num).name))


Closest detectors:
  n9
  na


We can also inspect the top two source classifications, ranked according to probability

In [5]:
print("\nTop 2 classifications (this calculation):")
rank = c.rankings()[:2]
for r in c.rankings()[:2]:
    print("  prob = {:.2f} {}".format(r[1], r[0].name))


Top 2 classifications (this calculation):
  prob = 0.95 GRB
  prob = 0.04 GENERIC_TRANSIENT


For comparison, let's also show the classifications determined by the flight software on-board the spacecraft

In [6]:
fsw_loc = trigdat.fsw_locations[-1]
print("\nTop 2 classifications (flight):")
for c, r in [fsw_loc.top_classification, fsw_loc.next_classification]:
    print("  prob = {:.2f} {}".format(r, c))


Top 2 classifications (flight):
  prob = 0.95 GRB
  prob = 0.04 TRANSNT


Look, they match!